The Coursera IBM Data Science Professional Certificate course provides an hands-on understanding of Dat science, Machine Learning using Cloud Tools: Jupyter / JupyterLab, GitHub, R Studio, and Watson Studio, API such as Foursquare and 
Libraries: Pandas, NumPy, Matplotlib, Seaborn, Folium, ipython-sql, Scikit-learn, ScipPy, etc.

The Capstone project is final project which carries the highest weightage.

## Capstone Project- Where to set up a Restaurant in Downtown Toronto.

### The Project Summary- 
These sections will be covered in the Project Report.
1. Introduction/Business Problem- Introduction is the section to discuss the business problem and who would be interested in this project.
2. Data- This section describes the data that will be used to solve the problem and the source of the data.
3. Methodology- This section represents the main component of the report to discuss and describe any exploratory data analysis, inferential statistical testing performed, and Machine learnings used and why.
4. Results- This is section to discuss the results.
5. Discussion- This is section to discuss any observations you noted and any recommendations you can make based on the results.
6. Conclusion- This is section to conclude the report.

<b> 1. Introduction/Business Problem-</b> A description of the problem and a discussion of the background.

An entrepreneur wants to open an restaurant in the Downtown Toronto area. There are several Neighborhoods in the Downtown Toronto area. The ideal location will be the one that already has different types of restaurants and also satifies other business criteria such as the target customers, type of the restaurant and finances available. The objective is to come up with the top 10 types of Venues in each neighborhood. This analysis will help any entreprenuer to determine the ideal location depending on the target customers, type of the restaurant and finances available for the business that is proposed to be opened.


<b>2. Data-</b> A description of the data and how it will be used to solve the problem.

The purpose of data collection is to enable to decide the top10 Establishments/ Venues in each neighborhood. So a getting a list of the Venues for each neighborhood in the Downtown Toronto area will enable us to find the top 10 Venues.
The following data is identified as necessary to help in achieving the business objective.
1. The list of postal code and Neighborhoods of Canada along with the Latitude and Longitude- This data will help in using the latitude and longitude to retrieve list of venues in each Neighborhood.

    a. The Wikipedia page has a list of Canada postal codes and Boroughs and Neighborhoods. This data will have to be scrapped from the web page. This data will be celaned up and formatted using one hot encoding techniques and pandas string replacement functions.

    b. Retrieve / Get the list of Geo coordinates (Latitude and Longitude) for each postal code. Alternatively, use a csv file that has all the Geo coordinates. This data will be merged with the data retrieved in step1.
    

2. The list of venues for each neighborhood- This data will help in using Machine Learning to find patterns in the data by Segmentation and Clustering.

    a. Use the Foursuare API to get the list of Venues for each Latitude and Longitude.


<b>3. Methodology-</b> Discuss and describe any exploratory data analysis, inferential statistical testing and Machine learning techniques used.
1. The Wikipedia page has a list of Canada postal codes and Boroughs and Neighborhoods. This data will have to be scrapped from the web page. This data will be cleaned and formatted using one hot encoding techniques and pandas string replacement functions.
2. Retrieve / Get the list of Geo coordinates (Latitude and Longitude) for each postal code. Alternatively, use a csv file that has all the Geo coordinates. This data will be merged with the data retrieved in step1.
3. Retrieve a list of Venues in each neighborhood. Use the Foursuare API to get the list of Venues for each Latitude and Longitude. An personal account was opened which will enable to get the data required for this purpose.
4. Construct a normalized Dataframe by converting the Categorical Values of the Venue categories to columns having numeric values.
5. Use Segmentaion and Clustering Technique to find the clusters which are similar.  Kmeans clustering will be used for this purpose as this is one of the effective techniques avaliable for this purpose.
6. Visualize the clusters by plotting a map. Folium library is a very good library avaliable for this purpose.
7. These clusters will let the Entreprenuer know the primary activity in that cluster and help him to decide the location. For ex. if the Entreprenuer wants to service office employees, the restaurant can be opened in a neighborhood which has business activity.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install geopy
#!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 94 kB 4.2 MB/s eta 0:00:011
Libraries imported.


### Scrape the data from the Wikipedia URL and create the pandas data frame with the Canada postal code and niegborhood data.

In [2]:
 # Scraping the data.
import requests
import lxml.html as lh
import pandas as pd
 from urllib.request import urlopen
 url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
 #Create a handle, page, to handle the contents of the website
 page = requests.get(url)
 #Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]
print ('len(T)',[len(T) for T in tr_elements[:12]])
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"' %(i,name))
    col.append((name,[]))
    
    
    #Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        #print ('len(T)!=3')
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        #print ('in for')
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        
        [len(C) for (title,C) in col]
        #print ('col', col)     

    

len(T) [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


In [3]:
# Create the Dataframe from the col array
Dict={title:column for (title,column) in col}
#print ('Dict', Dict)
df=pd.DataFrame(Dict)
df = df.replace({'\n':''}, regex=True)
df.columns=df.columns.str.replace('\n','')
df = df[df['Borough'] != 'Not assigned']
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', 1, df['Neighbourhood'])
df.head()


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
# Create a Canada postal csv file from the dataframe and store in IBM watson object storge.
#df.to_csv('canpostalcd')
from project_lib import Project
project =Project(project_id='6c00565f-d54b-4419-8ea0-5422c90cb00a', project_access_token='p-4c2fc97e8b68c62358d8689ba87a5c0548556576')
project.save_data(file_name = "canpostalcd.csv",data = df.to_csv(index=False), overwrite=True)
print ('canpostalcd csv created')

canpostalcd csv created


## use Geocode to retreive the latitude and the longitudes.

In [5]:
#This was done using the csv file because the geocode was not responding and returning data even after several attempts. 
#So, the code is commented.
#!pip install geocoder
#import geocoder # import geocoder
# initialize your variable to None
#lat_lng_coords = None
# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('North York, CA')
#  lat_lng_coords = g.latlng
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print ('lat {}, lon {}'.format(latitude,longitude))
# import geocoder
# g = geocoder.google('Mountain View, CA')
# g.latlng

### Data frame with latitude and longitudes of Canada postal codes.
This was done using the csv file because the geocode was not responding and returning data even after several attempts.

In [7]:
# Read the canada postal geo codes from csv file
df_can = pd.read_csv('https://cocl.us/Geospatial_data',   )
print ('Can geo codes Data read into a pandas dataframe!')

Can geo codes Data read into a pandas dataframe!


In [10]:
df_can.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# Merge the origial dataframe created using data scrapped from the wikipedia site.
# The how =left parameter ensures that postal code data is maintained in the new dataframe even if the lat, lon match is not found for the postal code in the geo csv file.
dfinal = df.merge(df_can, on="Postal Code", how="left")
dfinal.columns=dfinal.columns.str.replace('Neighbourhood','Neighborhood')
dfinal.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
# Create a Canada postal and geocode csv file from the dataframe and store in IBM watson object storge.
from project_lib import Project
project =Project(project_id='6c00565f-d54b-4419-8ea0-5422c90cb00a', project_access_token='p-4c2fc97e8b68c62358d8689ba87a5c0548556576')
project.save_data(file_name = "canpostalgeocd.csv",data = dfinal.to_csv(index=False), overwrite=True)
print ('csv can geocode created')  

csv can geocode created


### Explore and cluster the neighborhoods in Downtown Toronto.

In [11]:
# check the original dataframe
neighborhoods = dfinal
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 104 neighborhoods.


In [13]:
# Slice the original dataframe and create a new dataframe of the Downtown Toronto data.
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Downtown Toronto')].reset_index(drop=True)
toronto_data.columns=toronto_data.columns.str.replace('Neighbourhood','Neighborhood')
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [14]:
project.save_data(file_name = "canpostalgeocd-dn.csv",data = toronto_data.to_csv(index=False), overwrite=True)
print ('csv canpostalgeocd-dn created')  

csv canpostalgeocd-dn created


In [15]:
# Create the foursqare API request URL. The client_id and client_secret were created by opening a foursquare account.
CLIENT_ID = 'X5RIPZ3YIBUWAC2RXUW4Z1Z2QMMUCAX5BS14VOZYWMWXZZCW' # your Foursquare ID
CLIENT_SECRET = 'Z0ALVD55KE4MOP5R2OMSO0IWY3CQNLTLTP1EGDIPMWNEINHT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# Function to get the categories for all latitude and longitudes.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('name:', name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    print('venues_list:', venues_list)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Your credentails:
CLIENT_ID: X5RIPZ3YIBUWAC2RXUW4Z1Z2QMMUCAX5BS14VOZYWMWXZZCW
CLIENT_SECRET:Z0ALVD55KE4MOP5R2OMSO0IWY3CQNLTLTP1EGDIPMWNEINHT


In [20]:
dn_toronto_data_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

name: Regent Park, Harbourfront
name: Queen's Park, Ontario Provincial Government
name: Garden District, Ryerson
name: St. James Town
name: Berczy Park
name: Central Bay Street
name: Christie
name: Richmond, Adelaide, King
name: Harbourfront East, Union Station, Toronto Islands
name: Toronto Dominion Centre, Design Exchange
name: Commerce Court, Victoria Hotel
name: University of Toronto, Harbord
name: Kensington Market, Chinatown, Grange Park
name: CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
name: Rosedale
name: Stn A PO Boxes
name: St. James Town, Cabbagetown
name: First Canadian Place, Underground city
name: Church and Wellesley
venues_list: [[('Regent Park, Harbourfront', 43.6542599, -79.3606359, 'Roselle Desserts', 43.653446723052674, -79.3620167174383, 'Bakery'), ('Regent Park, Harbourfront', 43.6542599, -79.3606359, 'Tandem Coffee', 43.65355870959944, -79.36180945913513, 'Coffee Shop'), ('Regent Park, Harbourfront', 

In [21]:
print(dn_toronto_data_venues.shape)
dn_toronto_data_venues.head()

(1226, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [22]:
# Save the final dataframe as csv so that it can be used in the capstone final project.
project.save_data(file_name = "candntorontovenues.csv",data = dn_toronto_data_venues.to_csv(index=False),overwrite=True)
print ('csv candntorontovenues created')

csv candntorontovenues created


In [23]:
dn_toronto_data_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [24]:
print('There are {} uniques categories.'.format(len(dn_toronto_data_venues['Venue Category'].unique())))

There are 207 uniques categories.


In [26]:
# one hot encoding- This is to convet the Categories in to columns havinf numeric values.
dn_toronto_onehot = pd.get_dummies(dn_toronto_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dn_toronto_onehot['Neighborhood'] = dn_toronto_data_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dn_toronto_onehot.columns[-1]] + list(dn_toronto_onehot.columns[:-1])
dn_toronto_onehot = dn_toronto_onehot[fixed_columns]

dn_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [27]:
dn_toronto_onehot.shape

(1226, 207)

In [28]:
dn_toronto_grouped = dn_toronto_onehot.groupby('Neighborhood').mean().reset_index()
dn_toronto_grouped.shape

(19, 207)

In [28]:
num_top_venues = 5

for hood in dn_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dn_toronto_grouped[dn_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2          Restaurant  0.03
3  Seafood Restaurant  0.03
4              Bakery  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3     Boat or Ferry  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.19
1                Café  0.06
2      Sandwich Place  0.05
3  Italian Restaurant  0.05
4     Bubble Tea Shop  0.03


----Christie----
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3  Italian Restaurant  0.06
4          Baby Store  0.06


----Church and Wellesley----
                  venue  freq
0           Coffee Shop  0.07
1   Japanese Restaurant  0.06
2      Sushi Restaurant  0.06
3  F

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dn_toronto_grouped['Neighborhood']

for ind in np.arange(dn_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dn_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Bakery,Restaurant,Eastern European Restaurant,Liquor Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina,Boutique,Rental Car Location,Coffee Shop,Bar,Sculpture Garden
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Miscellaneous Shop,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Candy Store,Restaurant,Baby Store,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Mediterranean Restaurant,Men's Store,Café,Pizza Place


# Clustering Kmeans

In [30]:
# set number of clusters
kclusters = 5 # This will create 5 clusters

dn_toronto_grouped_clustering = dn_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dn_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 0, 4, 2, 2, 2, 2, 2, 2], dtype=int32)

In [42]:
# add clustering labels
# labels are already inserted in the above code.
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
dn_toronto_merged = toronto_data

# merge dn_toronto_merged with toronto_data to add latitude/longitude for each neighborhood
dn_toronto_merged = dn_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dn_toronto_merged.head() # check the last columns!
#dn_toronto_merged['Cluster Labels'].unique

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Theater,Wine Shop,Electronics Store,Performing Arts Venue
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Bar,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Hotel,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Lingerie Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,American Restaurant,Creperie,Cheese Shop,Restaurant,Hotel,Seafood Restaurant,Farmers Market
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Bakery,Restaurant,Eastern European Restaurant,Liquor Store


In [31]:
# Get the Latitude and longitude of Downtown Toronto, Country code CA
address = 'Downtown Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
dn_toronto_latitude = location.latitude
dn_toronto_longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(dn_toronto_latitude, dn_toronto_longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [44]:
# create map
map_clusters = folium.Map(location=[dn_toronto_latitude, dn_toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dn_toronto_merged['Latitude'], dn_toronto_merged['Longitude'], dn_toronto_merged['Neighborhood'], dn_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Print the Clusters

In [56]:
# get the 2nd column (index=1) and columns from 5 to the total number of columns (dn_toronto_merged.shape[1])
# do this for all 5 clusters.
print('The Cluster1')
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 0, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Theater,Wine Shop,Electronics Store,Performing Arts Venue
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Bar,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Distribution Center
2,"Garden District, Ryerson",0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Hotel,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Lingerie Store
3,St. James Town,0,Coffee Shop,Café,Cocktail Bar,American Restaurant,Creperie,Cheese Shop,Restaurant,Hotel,Seafood Restaurant,Farmers Market
4,Berczy Park,0,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Bakery,Restaurant,Eastern European Restaurant,Liquor Store
5,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Miscellaneous Shop,Japanese Restaurant
7,"Richmond, Adelaide, King",0,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega,Thai Restaurant,Gym,Sushi Restaurant,Clothing Store,Burrito Place
8,"Harbourfront East, Union Station, Toronto Islands",0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Sporting Goods Shop
9,"Toronto Dominion Centre, Design Exchange",0,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,Salad Place,Italian Restaurant,American Restaurant,Deli / Bodega
10,"Commerce Court, Victoria Hotel",0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant,Italian Restaurant


In [55]:
print('The Cluster2')
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 1, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,1,Park,Trail,Playground,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [54]:
print('The Cluster3')
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 2, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",2,Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina,Boutique,Rental Car Location,Coffee Shop,Bar,Sculpture Garden


In [53]:
print('The Cluster4')
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 3, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"University of Toronto, Harbord",3,Café,Bar,Italian Restaurant,Japanese Restaurant,Bakery,Bookstore,Yoga Studio,Sandwich Place,College Gym,Beer Store


In [52]:
print('The Cluster5')
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 4, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,4,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Candy Store,Restaurant,Baby Store,Coffee Shop


<b>4. Results-</b>

The Results display the top 10 most common venues for each Neighborhood. 
As can be seen from the results the Cluster1 has restaurants as all the top10 venues.
We used K-means clustering to group the Neighborhoods in to Clusters.
The map shows that the Cluster1 (red in color) has Neighborhoods spread out through out the Downton Toronto area.
The Results were generated by using data on 1/9/2021.


<b>5. Discussion-</b> As can be seen from the results the Cluster1 has restaurants as all the top10 venues. There are lot of neighborhood choices in Cluster1 to open a restaurant.
The other factors that would affect the decision are:
1. The Type/ Cuisine of the restaurant to be opened.
2. The customer profile that the restaurant wants to serve, for ex if the restaurant wants to target college students, Cluster4 which has university students will be a suitable location.
3. The budget of the business, this means factors such as rent and other costs have to be considered.
4. Finally, some other factors affecting the decision would be familiarity of the Entreprenuer with the Neighborhood, closeness of the Neighborhood for the Entreprenuer.

<b>6. Conclusion-</b>

The Report defines the Business problem and then identifies the data necessary that will help in making Recommendations.
The analysis is done by formatting and narrowing the data down to top10 venues for each neighborhood and using K-means clustering which enables the identification of similar Neighborhoods.
These Results and also other factors mentioned in the Discussion section will be considered in making a Recommedation.
The Recommedation will definitely help the Entreprenuer in taking a good decision about the type of Resataurant and the location considering other factors such as the finances avaliable and sets the entreprenuer on the path to Success!.